In [ ]:
from math import sqrt
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from keras import backend as K
from numpy import concatenate
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import requests
import pandas as pd
import sqlite3


# conn =sqlite3.connect('sqlite_8.db')
# c = conn.cursor()

# c.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(c.fetchall())

# c.execute("SELECT * FROM syslogs ORDER BY timestamp DESC LIMIT 1")
# print(c.fetchall())


limit = '16'
bucket_size = 10
timing =90


variable = int(60*bucket_size)

df= pd.read_pickle("dataframe.gz")


def makeDataframe(df):
    import datetime

    atype = str(df.tail(1)['bucket'].values[0])
    ctype = str(df.head(1)['bucket'].values[0])

    sample_series = []

    end = datetime.datetime.strptime(atype, '%Y-%m-%d %H:%M:%S')
    # start = end - datetime.timedelta(minutes = 3000)
    start = datetime.datetime.strptime(ctype, '%Y-%m-%d %H:%M:%S')

    delta = datetime.timedelta(minutes=bucket_size)
    # start = datetime.datetime.strptime( start, '%Y-%m-%d %H:%M:%S' )
    # end = datetime.datetime.strptime( end, '%Y-%m-%d %H:%M:%S' )
    t = start
    while t <= end :
        sample_series.append(datetime.datetime.strftime( t, '%Y-%m-%d %H:%M:%S'))
        t += delta
    sample = pd.DataFrame(sample_series)
    sample =sample.rename(columns={0:"bucket"})

    sample.insert(1, 'msg_freq',0)

    merged_df = df.merge(sample, how = 'outer', on = ['bucket'])
    new_df = merged_df.sort_values('bucket')
    new_df = new_df.drop(columns = ['msg_freq_y']).fillna(0)

    new_df  = new_df.reset_index()
    new_df = new_df.iloc[:,1:]
    df = new_df.rename(columns={"msg_freq_x": "msg_freq"})
    
    return df



def cumulativeFreq(df):
    df_cs = df.copy()
    df_cs['bucket'] = pd.to_datetime(df_cs['bucket'])
    df_cs["msg_freq"] = df_cs["msg_freq"].iloc[:].groupby([df_cs['bucket'].dt.hour, df_cs['bucket'].dt.date]).cumsum()
    
    return df_cs



def Normalizer(df):
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import MinMaxScaler

    df1 = df.iloc[:,1:]   #change required
#     values = dff.values
#     values = values.astype('float32')
#     # normalize features
#     scaler = MinMaxScaler(feature_range=(0, 1))
#     scaled = scaler.fit_transform(values)
#     dff = pd.DataFrame(scaled, columns=df.columns[1:]) #change required
    dff = df1/df1.max()
    dff.insert(0, 'bucket', df["bucket"]) #change required
    
    return dff


def sigmoidDataframe(df):
    
    def sigmoid(x):
        a=0.7
        return 1/(1+5*np.exp(-a*x))
    
    dff = df.copy()
    dff['msg_freq'] = sigmoid(dff['msg_freq'])
    return dff

df = makeDataframe(df)
df_cs = cumulativeFreq(df)

dfn = Normalizer(df)
dfn_cs = Normalizer(df_cs)



#Creating a synthesized dataset



import datetime
sample_series = []
x = np.array([[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55]])
z =x
# z=np.array([])
for i in range(110):
    y =np.random.rand(x.shape[1])*30
#     z=np.append(np.append(z,y),x)
#     z=np.append(z,x)
    z=np.append(z,x+y)

dfs = pd.DataFrame(z)

zeit ='2022-01-10 18:00:00'

end = datetime.datetime.strptime(zeit, '%Y-%m-%d %H:%M:%S')
start = end - datetime.timedelta(minutes = int((len(dfs)-1)*10))
# start = datetime.datetime.strptime(ctype, '%Y-%m-%d %H:%M:%S')

delta = datetime.timedelta(minutes=10)

t = start
while t <= end :
    sample_series.append(datetime.datetime.strftime( t, '%Y-%m-%d %H:%M:%S'))
    t += delta
syn_df = pd.DataFrame(sample_series)
syn_df =syn_df.rename(columns={0:"bucket"})

syn_df['values']=dfs

syn_dfn = Normalizer(syn_df)


Values = df_cs.iloc[:,1].values

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)



# choose a number of time steps; basically the memory of the network
n_steps = int(timing/bucket_size)

# split into samples
X, y = split_sequence(Values, n_steps)

n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))


def wisesplit(num,div):
    
    part = num - div
    while part/num > 0.8:
        part = part-div
    
    return part

def print_factors(num):
    import numpy as np
    factors = []
    for i in range(1, num + 1):
        if num % i == 0:
            factors.append(i)
    return np.array(factors)

arr =print_factors(len(X))
div = arr[int(0.6*arr.shape[0])]

split = wisesplit(len(X),div)
# split =int(0.8*len(X))
# split = -272

train_X = np.array(X)[:split]
train_y = np.array(y)[:split]
test_X = np.array(X)[split:]
test_y = np.array(y)[split:]

train_y =train_y.reshape((train_y.shape[0],1))

test_y = test_y.reshape((test_y.shape[0],1))

def compute_hcf(x, y):

# choose the smaller number
    if x > y:
        smaller = y
    else:
        smaller = x
    for i in range(1, smaller+1):
        if((x % i == 0) and (y % i == 0)):
            hcf = i 
    return hcf

batchsize = compute_hcf(train_X.shape[0],test_X.shape[0])
batch_arr = print_factors(batchsize)

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)


def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))



def Model(lr,epoch, batchsize):
    model = Sequential()
    model.add(LSTM(128, batch_input_shape=( batchsize,n_steps, n_features),return_sequences=False, stateful=True))
#     model.add(LSTM(96,return_sequences=True))
#     model.add(LSTM(64))
    model.add(Dense(1))
    opt = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss='mean_squared_logarithmic_error')

    print(model.summary())

    
    history = model.fit(x=train_X, y=train_y, epochs=epoch, validation_data=(test_X,test_y), batch_size=batchsize, verbose =1, shuffle=False)
    return model, history


lr_array = np.array([0.0001])
epoch_array = np.array([200])
batch_array = batch_arr[-1:]
print(batch_array)

for lr in lr_array:
    for ep in epoch_array:
        for bt in batch_array:
            
            print("\n\nNumber of time steps: {}".format(n_steps))
            print("Time bucket size: {}".format(bucket_size))
            print("Number of Epochs = {}".format(ep))
            print("Learning Rate = {}".format(lr))
            print("Batch size = {}\n\n".format(bt))
            
            model,history = Model(lr,ep,bt)

            y_train_pred = model.predict(train_X, batch_size=bt)
            y_train_pred = y_train_pred.reshape(y_train_pred.shape[0],1)
            train_mae_loss = np.abs(y_train_pred - train_y)

            threshold = np.amax(train_mae_loss)
            print(threshold)


            y_test_pred = model.predict(test_X, batch_size=bt)
            y_test_pred = y_test_pred.reshape(y_test_pred.shape[0],1)
            test_mae_loss = np.abs(y_test_pred - test_y)


            test_score_df = pd.DataFrame(test_mae_loss)
            test_score_df['threshold'] = threshold
            test_score_df['anomaly'] = test_score_df[0] > test_score_df['threshold']
            anomalies = test_score_df.loc[test_score_df['anomaly'] == True]
            print(anomalies.shape)

            test_anom = pd.DataFrame(test_y)
            pred_df = pd.DataFrame(y_test_pred)



            #PLOTTING

            plt.figure(figsize=(12,8))   
            plt.plot(history.history['loss'],label='Training loss')
            plt.plot(history.history['val_loss'], label ="Validation loss")
            plt.xlabel('Epochs', fontsize=10)
            plt.ylabel('Loss', fontsize=10)
#             plt.ylim(0,0.2)
            plt.legend(['Training loss', 'Validation loss'])
            plt.savefig('1e-4_10_90_layer1.pdf', dpi=250)
            plt.show()
            


#             plt.figure()    
#             plt.plot(history.history['acc'],label='Training  accuracy')
#             plt.plot(history.history['val_acc'], label ="Validation accuracy")
#             plt.show()


            fig = go.Figure()    
            fig.add_trace(go.Scatter(x=dfn_cs['bucket'][split+n_steps:], y= test_anom[0], name='actual'))
            fig.add_trace(go.Scatter(x=dfn_cs['bucket'][split+n_steps:], y= pred_df[0], name= 'prediction',line=dict(dash ='dot')))
            fig.update_layout(showlegend=True, title='Frequency v/s Time',width=1600, height=800)
            fig.update_yaxes(title_text='Normalized frequency')
            fig.update_xaxes(title_text='Time')
            fig.update_layout(xaxis_range=['2022-01-09 12:00:00','2022-01-10 00:00:00'])
#             fig.update_layout(yaxis_range=[-0.1,1.1])
#             fig.update_xaxes(title_text='Timestamp')
            fig.show()
            fig.write_image("p_1e-4_10_90_layer1.pdf")

#             fig = go.Figure()    
#             fig.add_trace(go.Scatter(x=syn_df['bucket'][split+n_steps:], y= np.abs(test_anom[0]-pred_df[0])/test_anom[0], name='error'))
#             fig.update_layout(showlegend=True, title='Absolute Error for test data')
#             fig.show()


